## Create final data (final_data)
### Create a dataframe with a single row per month, containing a column for each bin (with total acres) and columns for mean aqi, max aqi, and min aqi.  Write to a table named final_data.

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

%matplotlib inline


In [2]:
rds_connection_string = "root:12345678@127.0.0.1/fires"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')


In [3]:
data = pd.read_sql("select * from binned_by_acres", con=engine)
data.head()

/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


,index,date,overall_aqi,fire_id,size,distance,bins
0,0,2001-01-01,153,8659.0,0.1,380.928215,bigger then 300
1,1,2001-01-01,153,9387.0,0.1,136.804378,within 150
2,2,2001-01-01,153,17163.0,0.1,251.976491,within 300
3,3,2001-01-01,153,30114.0,0.1,200.715760,within 250
4,4,2001-01-01,153,30201.0,0.1,467.441792,bigger then 300


In [4]:
data = data[['date', 'overall_aqi', 'fire_id', 'size', 'distance', 'bins']]
data.head()

,date,overall_aqi,fire_id,size,distance,bins
0,2001-01-01,153,8659.0,0.1,380.928215,bigger then 300
1,2001-01-01,153,9387.0,0.1,136.804378,within 150
2,2001-01-01,153,17163.0,0.1,251.976491,within 300
3,2001-01-01,153,30114.0,0.1,200.715760,within 250
4,2001-01-01,153,30201.0,0.1,467.441792,bigger then 300


## Group by date and bins (ie create total acres per bin per month)
* Note: because we only count acres burned on its end date, we need to group our dates by something larger then a day

In [5]:
# data.groupby(['bins','fire_id'])['size'].sum()

In [6]:
# test_group = data.groupby(pd.Grouper(key="date", freq="M"))['size'].sum()

In [7]:
grouped_acres = data.groupby([pd.Grouper(key="date", freq="M"),'bins'])['size'].sum()

In [8]:
type(grouped_acres)

pandas.core.series.Series

In [9]:
grouped_acres.head(30)

date        bins           
2001-01-31  bigger then 300    10399.4
            within 100             0.1
            within 150            40.3
            within 200            20.6
            within 250            86.5
            within 300             4.0
            within 50              0.1
2001-02-28  bigger then 300        6.3
            within 100             1.2
            within 150             1.3
            within 200             0.1
            within 250            76.4
            within 300             0.1
            within 50              0.2
2001-03-31  bigger then 300      202.3
            within 100             0.8
            within 150            54.7
            within 200            12.5
            within 250            15.7
            within 300             2.2
            within 50              5.0
2001-04-30  bigger then 300       33.1
            within 100             1.2
            within 150            21.6
            within 200            19

In [10]:
grouped_acres = grouped_acres.to_frame().reset_index().pivot(index = "date",columns = "bins", values = "size")

In [11]:
grouped_acres.head()

bins,bigger then 300,within 100,within 150,within 200,within 250,within 300,within 50
date,,,,,,,
2001-01-31,10399.4,0.1,40.3,20.6,86.5,4.0,0.1
2001-02-28,6.3,1.2,1.3,0.1,76.4,0.1,0.2
2001-03-31,202.3,0.8,54.7,12.5,15.7,2.2,5.0
2001-04-30,33.1,1.2,21.6,19.0,13.3,4.5,NaN
2001-05-31,109.8,68.9,1156.0,526.5,988.1,1963.1,0.3


In [12]:
grouped_acres = grouped_acres.fillna(0)

In [13]:
grouped_acres.head()

bins,bigger then 300,within 100,within 150,within 200,within 250,within 300,within 50
date,,,,,,,
2001-01-31,10399.4,0.1,40.3,20.6,86.5,4.0,0.1
2001-02-28,6.3,1.2,1.3,0.1,76.4,0.1,0.2
2001-03-31,202.3,0.8,54.7,12.5,15.7,2.2,5.0
2001-04-30,33.1,1.2,21.6,19.0,13.3,4.5,0.0
2001-05-31,109.8,68.9,1156.0,526.5,988.1,1963.1,0.3


## Group by air quality-- max, min, and avg

In [14]:
max_aqi = data.groupby([pd.Grouper(key="date", freq="M")])['overall_aqi'].max()

In [15]:
max_aqi = max_aqi.to_frame()
max_aqi.columns = ['max_aqi']
max_aqi.head()

,max_aqi
date,
2001-01-31,158
2001-02-28,117
2001-03-31,66
2001-04-30,49
2001-05-31,67


In [16]:
min_aqi=data.groupby([pd.Grouper(key="date", freq="M")])['overall_aqi'].min()

In [17]:
min_aqi = min_aqi.to_frame()
min_aqi.columns = ['min_aqi']
min_aqi.head()

,min_aqi
date,
2001-01-31,25
2001-02-28,30
2001-03-31,22
2001-04-30,23
2001-05-31,24


In [18]:
min_aqi.head()

,min_aqi
date,
2001-01-31,25
2001-02-28,30
2001-03-31,22
2001-04-30,23
2001-05-31,24


In [19]:
mean_aqi=data.groupby([pd.Grouper(key="date", freq="M")])['overall_aqi'].mean()

In [20]:
mean_aqi = mean_aqi.to_frame()
mean_aqi.columns = ['mean_aqi']
mean_aqi.head()

,mean_aqi
date,
2001-01-31,100.569620
2001-02-28,44.411765
2001-03-31,39.555556
2001-04-30,37.675325
2001-05-31,39.112532


## Merge them

In [21]:
grouped_acres.head()

bins,bigger then 300,within 100,within 150,within 200,within 250,within 300,within 50
date,,,,,,,
2001-01-31,10399.4,0.1,40.3,20.6,86.5,4.0,0.1
2001-02-28,6.3,1.2,1.3,0.1,76.4,0.1,0.2
2001-03-31,202.3,0.8,54.7,12.5,15.7,2.2,5.0
2001-04-30,33.1,1.2,21.6,19.0,13.3,4.5,0.0
2001-05-31,109.8,68.9,1156.0,526.5,988.1,1963.1,0.3


In [22]:
all_data = pd.merge(grouped_acres, mean_aqi, on="date")

In [23]:
all_data = pd.merge(all_data, max_aqi, on="date")

In [24]:
all_data = pd.merge(all_data, min_aqi, on="date")

In [25]:
all_data.head()

,bigger then 300,within 100,within 150,within 200,within 250,within 300,within 50,mean_aqi,max_aqi,min_aqi
date,,,,,,,,,,
2001-01-31,10399.4,0.1,40.3,20.6,86.5,4.0,0.1,100.569620,158,25
2001-02-28,6.3,1.2,1.3,0.1,76.4,0.1,0.2,44.411765,117,30
2001-03-31,202.3,0.8,54.7,12.5,15.7,2.2,5.0,39.555556,66,22
2001-04-30,33.1,1.2,21.6,19.0,13.3,4.5,0.0,37.675325,49,23
2001-05-31,109.8,68.9,1156.0,526.5,988.1,1963.1,0.3,39.112532,67,24


## Write to database

In [26]:
all_data.to_sql(name="final_data", con=engine, if_exists="replace", index=True)